In [4]:
import cv2
import numpy as np
import os

def process_image(image_path, output_folder):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return

    # 1. Preprocessing (Convert to Grayscale and Blur)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Adjust kernel size as needed

    # 2. Binarization (Adaptive Thresholding)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY_INV, 11, 2)  # Adjust block size and C

    # 3. Morphological Operations (Closing)
    kernel = np.ones((3, 3), np.uint8)  # Adjust kernel size as needed
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)  # Adjust iterations

    # 4. Segmentation and Contour Extraction
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by area (largest first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    mask = np.zeros_like(gray)

    num_contours_to_draw = min(3, len(contours))

    for i in range(num_contours_to_draw):
        cv2.drawContours(mask, contours, i, (255), thickness=cv2.FILLED)

    extracted_shapes = cv2.bitwise_and(image, image, mask=mask)

    output_filename = f"shapes_{os.path.basename(image_path)}"
    output_path = os.path.join(output_folder, output_filename)
    cv2.imwrite(output_path, extracted_shapes)
    print(f"Shapes extracted from {image_path}. Saved as {output_path}")


    background = np.ones_like(image, dtype=np.uint8) * 255
    shapes_on_white = np.where(mask[..., None] == 255, extracted_shapes, background).astype(np.uint8)
    output_filename_white = f"shapes_white_{os.path.basename(image_path)}"
    output_path_white = os.path.join(output_folder, output_filename_white)
    cv2.imwrite(output_path_white, shapes_on_white)



def process_images_in_folder(folder_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            process_image(image_path, output_folder)


# Example usage:
input_folder = "/home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST"  # Replace with your input folder
output_folder = "/home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED"  # Replace with your desired output folder
process_images_in_folder(input_folder, output_folder)

Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/0.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_0.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/1.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_1.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/11.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_11.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/13.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_13.png
Shapes extracted from /home/ashwathama/Documents/GitHub/2dshapedetection-major/TEST/14.png. Saved as /home/ashwathama/Documents/GitHub/2dshapedetection-major/ALGO_IMAGE_PROCESSED/shapes_14.png
Shapes extracted from /home/ashwathama/

# PREPORCESSING HERE


In [1]:
import cv2
import numpy as np
import math

# Load and preprocess the image
img = cv2.imread(r'ElementaryCQT/Inscribed/Circle_triangle/circ_tri_3.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1)
edges = cv2.Canny(blurred, 50, 150)

# Find contours and hierarchy
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# simplify the hierarchy array
hierarchy = hierarchy[0]

shapes = []

# Loop through contours
for i, cnt in enumerate(contours):

    # Approximate contour to polygon (simplify shape)
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    vertices = len(approx)

    # calculate moments for center and area
    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    center = (cx, cy)

    area = cv2.contourArea(cnt)
    
    # get sides (distances between consecutive points)
    side_lengths = []
    for j in range(vertices):
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]
        dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        side_lengths.append(dist)

    # get angles
    angles = []
    for j in range(vertices):
        p0 = approx[j - 1][0]
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]

        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])

        dot_prod = v1[0]*v2[0] + v1[1]*v2[1]
        mag1 = math.hypot(v1[0], v1[1])
        mag2 = math.hypot(v2[0], v2[1])

        #angle = math.acos(dot_prod / (mag1 * mag2)) * 180 / math.pi
        if mag1 == 0 or mag2 == 0:
            angle = 0  # You can choose to skip it or set a default value
        else:
            cos_theta = dot_prod / (mag1 * mag2)
            # Clamp the cosine value to [-1, 1] to prevent math domain errors
            cos_theta = max(min(cos_theta, 1), -1)
            angle = math.acos(cos_theta) * 180 / math.pi


        angles.append(angle)

    # shape classification
    shape = "Unknown"
    
    # triangles
    if vertices == 3:
        shape = "Triangle"

    # quadrilateral checks
    elif vertices == 4:
        # check if sides are almost equal
        sides_equal = all(abs(side_lengths[k] - side_lengths[0]) < 5 for k in range(1, 4))
        # check all angles are close to 90 degrees
        angles_close_90 = all(abs(a - 90) < 10 for a in angles)

        if sides_equal and angles_close_90:
            shape = "Square"
        elif angles_close_90:
            shape = "Rectangle"
        else:
            shape = "Quadrilateral"

    # circles and others
    elif vertices > 4:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        circularity = 4 * math.pi * area / (cv2.arcLength(cnt, True) ** 2)

        if circularity > 0.8:
            shape = "Circle"
        else:
            shape = f"{vertices}-gon"

    # collect shape info
    shapes.append({
        'index': i,
        'shape': shape,
        'center': center,
        'area': area,
        'parent': hierarchy[i][3],
        'contour': approx,
        'sides': side_lengths,
        'angles': angles
    })

# check for inscribed shapes
for s in shapes:
    if s['parent'] != -1:
        parent = shapes[s['parent']]

        dx = abs(s['center'][0] - parent['center'][0])
        dy = abs(s['center'][1] - parent['center'][1])

        # centers aligned (adjust threshold as needed)
        if dx < 10 and dy < 10:
            area_ratio = s['area'] / parent['area']
            
            # rule-of-thumb area ratio for inscribed shapes
            if 0.3 < area_ratio < 0.7:
                print(f"{s['shape']} is inscribed in {parent['shape']}")

# draw all shapes
for s in shapes:
    cv2.drawContours(img, [s['contour']], -1, (0, 255, 0), 2)
    cv2.putText(img, s['shape'], s['center'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

cv2.imshow('Inscribed Shape Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


## Working 2d Shape Classifier

In [1]:
import cv2
import numpy as np
import math

# Load and preprocess the image
img = cv2.imread(r'C:\Users\ashut\Documents\GitHub\ProjectMajor\IMGTEST\003.png')

if img is None:
    print("Image not found or path is incorrect!")
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1)
edges = cv2.Canny(blurred, 50, 150)

cv2.imshow("Edges", edges)  # See if edges are being detected properly

# Find contours and hierarchy
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) == 0:
    print("No contours found!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

if hierarchy is None:
    print("No hierarchy returned!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# simplify the hierarchy array
hierarchy = hierarchy[0]

shapes = []

# Loop through contours
for i, cnt in enumerate(contours):

    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    vertices = len(approx)

    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    center = (cx, cy)

    area = cv2.contourArea(cnt)

    side_lengths = []
    for j in range(vertices):
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]
        dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        side_lengths.append(dist)

    angles = []
    for j in range(vertices):
        p0 = approx[j - 1][0]
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]

        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])

        dot_prod = v1[0]*v2[0] + v1[1]*v2[1]
        mag1 = math.hypot(v1[0], v1[1])
        mag2 = math.hypot(v2[0], v2[1])

        if mag1 == 0 or mag2 == 0:
            angle = 0
        else:
            cos_theta = dot_prod / (mag1 * mag2)
            cos_theta = max(min(cos_theta, 1), -1)
            angle = math.acos(cos_theta) * 180 / math.pi

        angles.append(angle)

    shape = "Unknown"
    
    if vertices == 3:
        shape = "Triangle"
    elif vertices == 4:
        sides_equal = all(abs(side_lengths[k] - side_lengths[0]) < 5 for k in range(1, 4))
        angles_close_90 = all(abs(a - 90) < 10 for a in angles)

        if sides_equal and angles_close_90:
            shape = "Square"
        elif angles_close_90:
            shape = "Rectangle"
        else:
            shape = "Quadrilateral"
    elif vertices > 4:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        circularity = 4 * math.pi * area / (cv2.arcLength(cnt, True) ** 2)

        if circularity > 0.8:
            shape = "Circle"
        else:
            shape = f"{vertices}-gon"

    #print(f"Detected: {shape}")
    print(shape)
    shapes.append({
        'index': i,
        'shape': shape,
        'center': center,
        'area': area,
        'parent': hierarchy[i][3],
        'contour': approx,
        'sides': side_lengths,
        'angles': angles
    })


'''  inscribed shape formatter '''

for s in shapes:
    if s['parent'] != -1:
        parent = shapes[s['parent']]
        dx = abs(s['center'][0] - parent['center'][0])
        dy = abs(s['center'][1] - parent['center'][1])
        area_ratio = s['area'] / parent['area']
        print(f"Checking: {s['shape']} inside {parent['shape']} -> dx: {dx}, dy: {dy}, area_ratio: {area_ratio}")
        if dx < 10 and dy < 10:
            if 0.3 < area_ratio < 0.7:
                print(f"{s['shape']} is inscribed in {parent['shape']}")

for s in shapes:
    cv2.drawContours(img, [s['contour']], -1, (0, 255, 0), 2)
    cv2.putText(img, s['shape'], s['center'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
cv2.imshow('shapes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Unknown
9-gon
7-gon
Circle
Circle
Square
Circle
7-gon
Circle
Circle
Circle
Circle
10-gon
10-gon
8-gon
14-gon
8-gon
13-gon
Circle
12-gon
Circle
9-gon
6-gon
14-gon
12-gon
Circle
6-gon
10-gon
13-gon
Circle
10-gon
7-gon
8-gon
16-gon
6-gon
8-gon
15-gon
6-gon
Circle
7-gon
9-gon
12-gon
9-gon
15-gon
10-gon
6-gon
6-gon
18-gon
Circle
Circle
10-gon
10-gon
Circle
6-gon
6-gon
6-gon
13-gon
Circle
10-gon
10-gon
6-gon
6-gon
6-gon
8-gon
Circle
16-gon
8-gon
6-gon
Square
10-gon
10-gon
Unknown
Unknown
7-gon
6-gon
7-gon
9-gon
11-gon
10-gon
9-gon
Circle
5-gon
15-gon
Circle
9-gon
7-gon
6-gon
6-gon
6-gon
10-gon
6-gon
10-gon
6-gon
6-gon
10-gon
6-gon
Circle
Circle
Unknown
15-gon
Square
9-gon
14-gon
9-gon
14-gon
9-gon
6-gon
7-gon
8-gon
Circle
9-gon
9-gon
Circle
8-gon
10-gon
6-gon
13-gon
10-gon
Quadrilateral
6-gon
Square
Circle
8-gon
9-gon
8-gon
6-gon
9-gon
Square
Circle
13-gon
9-gon
6-gon
9-gon
7-gon
7-gon
9-gon
7-gon
8-gon
Square
Circle
8-gon
9-gon
11-gon
11-gon
12-gon
8-gon
10-gon
6-gon
12-gon
Circle
10-gon
8-

In [ ]:
import cv2
import numpy as np
import math

#processing
img = cv2.imread(r'ElementaryCQT/Inscribed/Circle_triangle/circ_tri_3.png')
if img is None:
    print("Image not found or path is incorrect!")
    exit()


#-------------------------------------------------------------------------------------------------------------- preprocessing image

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1)
edges = cv2.Canny(blurred, 50, 150)
cv2.imshow("Edges", edges)

#-----------------------------------------------------------------================================================PREPROCESSING END--------------------------------

# === ================================================================================================= ====================================== contours =======
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
if not contours:
    print("No contours found!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

#large contour
contour = max(contours, key=cv2.contourArea)
epsilon = 0.02 * cv2.arcLength(contour, True)  #approx contour
approx = cv2.approxPolyDP(contour, epsilon, True)

#points list
point_list = [tuple(pt[0]) for pt in approx]
vertices_count = len(point_list)

#---------------------- Center (optional)--------------------------------
M = cv2.moments(contour)
cx, cy = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])) if M['m00'] != 0 else (0, 0)
center = (cx, cy)

# Area and Perimeter
area = cv2.contourArea(contour)
perimeter = cv2.arcLength(contour, True)

# ----------------------------------------- ------------------------------------------------------------------------------------ Shape classification---------------------
# ----------------------------------------- ------------------------------------------------------------------------------------ Shape classification---------------------
# ----------------------------------------- ------------------------------------------------------------------------------------ Shape classification---------------------
shape = "Unknown"
if vertices_count == 3:
    shape = "Triangle"
elif vertices_count == 4:
    side_lengths = []   #side lengths
    for j in range(4):
        p1 = point_list[j]
        p2 = point_list[(j + 1) % 4]
        length = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        side_lengths.append(length)

    
    angles = []         #get angles
    for j in range(4):
        #------------points----------------
        p0 = point_list[j - 1]
        p1 = point_list[j]
        p2 = point_list[(j + 1) % 4]
        
        #------------vectors----------------
        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])

        #------------angle----------------
        dot_prod = v1[0]*v2[0] + v1[1]*v2[1]
        mag1 = math.hypot(v1[0], v1[1])
        mag2 = math.hypot(v2[0], v2[1])

        if mag1 == 0 or mag2 == 0:
            angle = 0
        else:
            cos_theta = dot_prod / (mag1 * mag2)
            cos_theta = max(min(cos_theta, 1), -1)
            angle = math.acos(cos_theta) * 180 / math.pi
        angles.append(angle)

    #
    sides_equal = all(abs(side_lengths[k] - side_lengths[0]) < 5 for k in range(1, 4))
    angles_close_90 = all(abs(a - 90) < 10 for a in angles)

    if sides_equal and angles_close_90:
        shape = "Square"
    elif angles_close_90:
        shape = "Rectangle"
    else:
        shape = "Quadrilateral"

elif vertices_count > 4:
    circularity = 4 * math.pi * area / (perimeter ** 2) if perimeter != 0 else 0

    if circularity > 0.8:
        shape = "Circle"
    else:
        shape = f"{vertices_count}-gon"
print(shape)
cv2.imshow('images', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Circle


In [ ]:
import os
import cv2
import csv
import math
import numpy as np
import pandas as pd

# === Step 1: Load CSV Files ===
csv_files = {
    "GeoS": "GeoS_Processed.csv",
    "GeoQA": "GeoQA_Processed.csv",
    "Geometry3K": "Geometry3K_Processed.csv"
}

# Load datasets into Pandas DataFrames, handling errors
df_dict = {}
for dataset, file in csv_files.items():
    if os.path.exists(file):
        try:
            df_dict[dataset] = pd.read_csv(file, encoding="utf-8", on_bad_lines="skip", dtype={"folder_name": str})
        except Exception as e:
            print(f"❌ Error loading {file}: {e}")
    else:
        print(f"⚠️ CSV file not found: {file}")

# === Step 2: Shape Detection Using OpenCV ===
def detect_shape(image_path):
    """Detects the shape in the given image using contour approximation."""
    img = cv2.imread(image_path)

    if img is None:
        print(f"❌ Image not found: {image_path}")
        return None

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 1)

    # Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return "unknown"

    # Select the largest contour
    contour = max(contours, key=cv2.contourArea)

    # Approximate contour to polygon
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Count vertices
    vertices_count = len(approx)

    # Compute area & perimeter
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)

    # === Shape Classification ===
    shape = "unknown"

    if vertices_count == 3:
        shape = "triangle"
    elif vertices_count == 4:
        # Get angles to distinguish rectangle/square
        side_lengths = [math.hypot(approx[i][0][0] - approx[(i + 1) % 4][0][0], 
                                   approx[i][0][1] - approx[(i + 1) % 4][0][1]) for i in range(4)]

        angles = []
        for i in range(4):
            p0, p1, p2 = approx[i - 1][0], approx[i][0], approx[(i + 1) % 4][0]
            v1 = (p0[0] - p1[0], p0[1] - p1[1])
            v2 = (p2[0] - p1[0], p2[1] - p1[1])
            dot_prod = v1[0] * v2[0] + v1[1] * v2[1]
            mag1 = math.hypot(v1[0], v1[1])
            mag2 = math.hypot(v2[0], v2[1])
            cos_theta = dot_prod / (mag1 * mag2) if mag1 and mag2 else 0
            angle = math.acos(min(max(cos_theta, -1), 1)) * 180 / math.pi
            angles.append(angle)

        is_square = all(abs(side_lengths[i] - side_lengths[0]) < 5 for i in range(1, 4))
        is_rectangle = all(abs(a - 90) < 10 for a in angles)

        if is_square:
            shape = "square"
        elif is_rectangle:
            shape = "rectangle"
        else:
            shape = "quadrilateral"

    elif vertices_count > 4:
        circularity = 4 * math.pi * area / (perimeter ** 2) if perimeter != 0 else 0
        if circularity > 0.5:
            shape = "circle"
        else:
            shape = "unknown"  # Avoid predicting as polygon

    return shape

# === Step 3: Get Image Path Correctly ===
def get_image_path(row, dataset):
    """Finds the correct image path based on the dataset structure."""
    if dataset == "Geometry3K":
        base_path = os.path.dirname(row["image_path"])  # Extract the dataset path
        image_folder = str(row["folder_name"])  # Convert to string

        img_path_1 = os.path.join(base_path, image_folder, "img_diagram.png")
        img_path_2 = os.path.join(base_path, image_folder, "img_diagram_point.png")

        if os.path.exists(img_path_1):
            return img_path_1
        elif os.path.exists(img_path_2):
            return img_path_2
        else:
            return None  # No valid image found

    return row["image_path"]  # GeoS & GeoQA already have direct image paths

# === Step 4: Evaluate All Images & Save Results to CSV ===
results = []
output_csv = "Shape_Predictions.csv"

def evaluate_images():
    true_positive = 0
    false_positive = 0
    total_images = 0

    for dataset, df in df_dict.items():
        for _, row in df.iterrows():
            if row["shape"].lower() == "unknown":
                continue  # Skip unknown shapes

            # Extract image details
            expected_shape = row["shape"].lower()
            image_path = get_image_path(row, dataset)

            # Ensure the file exists before processing
            if image_path is None or not os.path.exists(image_path):
                print(f"⚠️ Image missing: {image_path}")
                continue  

            # Detect the shape in the image
            detected_shape = detect_shape(image_path)
            if detected_shape is None:
                continue  # Skip if detection failed

            # Compare expected vs detected shape
            is_correct = "Yes" if detected_shape == expected_shape else "No"

            # Update evaluation metrics
            total_images += 1
            if is_correct == "Yes":
                true_positive += 1
            else:
                false_positive += 1
            results.append([dataset, row["image_path"], expected_shape, detected_shape, is_correct])

    # Save results to CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["dataset", "image_path", "expected_shape", "shape_predicted", "is_shape_predicted"])
        writer.writerows(results)

    accuracy = (true_positive / total_images) * 100 if total_images > 0 else 0
    print(f"Total Images Evaluated: {total_images}")
    print(f"True Positives (Correct Predictions): {true_positive}")
    print(f"False Positives (Incorrect Predictions): {false_positive}")
    print(f"Accuracy: {accuracy:.2f}%")
    print("Results saved")

evaluate_images()


Total Images Evaluated: 3105
True Positives (Correct Predictions): 615
False Positives (Incorrect Predictions): 2490
Accuracy: 19.81%
Results saved


In [ ]:
import os
import cv2
import csv
import math
import numpy as np
import pandas as pd

# === Step 1: Load CSV Files ===
csv_files = {
    "GeoS": "GeoS_Processed.csv",
    "GeoQA": "GeoQA_Processed.csv",
    "Geometry3K": "Geometry3K_Processed.csv"
}

# Load datasets into Pandas DataFrames
df_dict = {}
for dataset, file in csv_files.items():
    if os.path.exists(file):
        try:
            df_dict[dataset] = pd.read_csv(file, encoding="utf-8", on_bad_lines="skip", dtype={"folder_name": str})
        except Exception as e:
            print(f"❌ Error loading {file}: {e}")
    else:
        print(f"⚠️ CSV file not found: {file}")



# === Step 2: Shape Detection Using OpenCV ===
def detect_shape(image_path):
    """Detects the shape in the given image using contour-based shape detection."""
    img = cv2.imread(image_path)

    if img is None:
        print(f"❌ Image not found: {image_path}")
        return "unknown"

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 1)
    
    # Edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return "unknown"

    # Select the largest contour
    contour = max(contours, key=cv2.contourArea)

    # Approximate contour to polygon
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Count vertices
    vertices_count = len(approx)

    # Compute area & perimeter
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)

    # === Shape Classification ===
    shape = "unknown"

    if vertices_count == 3:
        return "triangle"  # Always classify as triangle

    elif vertices_count == 4:
        return classify_quadrilateral(approx)  # Must be Square, Rectangle, Trapezium, or Parallelogram

    else:
        circularity = 4 * math.pi * area / (perimeter ** 2) if perimeter != 0 else 0
        if circularity > 0.5:
            return "circle"  # Circle detection remains independent

    return shape



# === Step 3: Quadrilateral Classification ===
def classify_quadrilateral(approx):
    """Classifies quadrilaterals as Square, Rectangle, Trapezium, or Parallelogram.
       If none of these match, returns 'unknown'. """

    side_lengths = [math.dist(approx[i][0], approx[(i + 1) % 4][0]) for i in range(4)]
    angles = []

    for i in range(4):
        p0, p1, p2 = approx[i - 1][0], approx[i][0], approx[(i + 1) % 4][0]
        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])
        dot_prod = v1[0] * v2[0] + v1[1] * v2[1]
        mag1 = math.dist(p0, p1)
        mag2 = math.dist(p1, p2)
        angle = math.degrees(math.acos(dot_prod / (mag1 * mag2))) if mag1 and mag2 else 0
        angles.append(angle)

    is_square = all(abs(side_lengths[i] - side_lengths[0]) < 5 for i in range(1, 4))
    is_rectangle = all(abs(a - 90) < 10 for a in angles)

    if is_square:
        return "square"
    elif is_rectangle:
        return "rectangle"
    elif abs(side_lengths[0] - side_lengths[2]) < 5 and abs(side_lengths[1] - side_lengths[3]) < 5:
        return "parallelogram"
    elif angles.count(90) == 1:
        return "trapezium"

    return "unknown"  # No "quadrilateral" output, only specific shapes or "unknown"



# === Step 4: Get Image Paths from CSV ===
def get_image_path(row, dataset):
    """Finds the correct image path based on dataset structure."""
    if dataset == "Geometry3K":
        base_path = os.path.dirname(row["image_path"])
        image_folder = str(row["folder_name"])

        img_path_1 = os.path.join(base_path, image_folder, "img_diagram.png")
        img_path_2 = os.path.join(base_path, image_folder, "img_diagram_point.png")

        return img_path_1 if os.path.exists(img_path_1) else img_path_2 if os.path.exists(img_path_2) else None

    return row["image_path"]



# === Step 5: Process & Save Results to CSV ===
output_csv = "Shape_PredictionsV5.csv"
results = []
dataset_accuracy = {}

def evaluate_images():
    """Processes all images in CSV, predicts shapes, and calculates accuracy for each dataset."""
    for dataset, df in df_dict.items():
        truePositives = 0
        falsePositive = 0
        totalImg = 0

        for _, row in df.iterrows():
            if row["shape"].lower() not in ["triangle", "square", "rectangle", "trapezium", "parallelogram", "circle"]:
                continue  # Skip other shapes

            image_path = get_image_path(row, dataset)

            if image_path is None or not os.path.exists(image_path):
                continue  

            detected_shape = detect_shape(image_path)
            if detected_shape is None or detected_shape == "unknown":
                continue  # Ignore cases where the detected shape is unknown

            is_correct = "Yes" if detected_shape == row["shape"].lower() else "No"

            # Update evaluation metrics
            totalImg += 1
            if is_correct == "Yes":
                truePositives += 1
            else:
                falsePositive += 1

            results.append([dataset, row["image_path"], row["shape"], detected_shape, is_correct])

       
        accuracy = (truePositives / totalImg) * 100 if totalImg > 0 else 0
        dataset_accuracy[dataset] = accuracy

        print(f"Accuracy for {dataset}: {accuracy:.2f}%")
        print(f"Total : {totalImg} \n")
#------------------------------------------------------------------------------------------------------------
    '''    Save results to CSV     '''
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["dataset", "image_path", "expected_shape", "shape_predicted", "is_shape_predicted"])
        writer.writerows(results)
    print(" Results saved ")




#main  ---------------
evaluate_images()


Accuracy for GeoS: 80.39%
Total : 51 

Accuracy for GeoQA: 69.31%
Total : 518 

Accuracy for Geometry3K: 90.69%
Total : 376 

 Results saved 
